In [1]:
# Imports 
import chessdotcom as chess
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
# Define player profile: warning it is case sensitive
player_profile = 'aledbs' 

In [3]:
def check_user(player_profile):
    try: 
        chess.get_player_game_archives(player_profile).json['archives']
        print("User profile is correct")
    except:
        print("Incorrect user profile") 

In [4]:
def get_player_timeline(player_profile:str):
    '''
    Get list of all years and months were the player played games on chess.com 

        Parameters:
            player_profile (str): Profile of the player
        
        Returns: 
            List of lists with Year and Month where player played
    '''
    list_archives_games = chess.get_player_game_archives(player_profile).json['archives']
    year_month_full = [i.split('/')[-2:] for i in list_archives_games]

    # Checking if existing data is already player's data, if so get only delta timeline for games played since last integration
    if os.path.isfile('./data/raw_api_data.pkl'):
        existing_game_df = pd.read_pickle('./data/raw_api_data.pkl')
        last_game_played_year = datetime.date.fromtimestamp(existing_game_df['end_time'].max()).year
        last_game_played_month = datetime.date.fromtimestamp(existing_game_df['end_time'].max()).month
        if existing_game_df['player_profile'].iloc[0] == player_profile:
            return [time for time in year_month_full if (int(time[0]) >= last_game_played_year) & (int(time[1]) >= last_game_played_month)]
        else:
            return year_month_full
    else:
        return year_month_full

In [29]:
def get_all_player_games(player_profile:str, player_timeline:list):
  '''
    Get list of all years and months were the player played games on chess.com 

        Parameters:
            player_profile: Profile of the player
            player_timeline: List of lists with years and months played
        
        Returns: 
            game_df (DataFrame): Df containing all games played by the player since day 1
  '''
  game_df = pd.DataFrame()

  for month_year in player_timeline:
      monthly_game = chess.get_player_games_by_month(player_profile, month_year[0], month_year[1])
      monthly_game_df = pd.DataFrame(monthly_game.json['games'])
      game_df = pd.concat([game_df,monthly_game_df])
    
  game_df['player_profile'] = player_profile

  if os.path.isfile('./data/raw_api_data.pkl'):
     existing_game_df = pd.read_pickle('./data/raw_api_data.pkl')
     game_df = pd.concat([existing_game_df, game_df])   

  game_df = pd.concat([game_df, game_df.accuracies.apply(pd.Series).add_suffix('_accuracie')], axis=1)

  game_df.pop('white')
  game_df.pop('black')
  game_df.pop('accuracies')
  game_df.pop('0_accuracie')
  game_df = game_df.drop_duplicates()

  return game_df

In [6]:
check_user(player_profile)

User profile is correct


In [7]:
player_timeline = get_player_timeline(player_profile)

In [30]:
game_df = get_all_player_games(player_profile, player_timeline)

In [31]:
game_df.to_pickle('./data/raw_api_data.pkl') 